In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import ClassificationModel, EmbeddingsModel, SiameseModel
from trainer import SiameseTrainerCombinationDataset, all_subjects_intense
import torch

import wandb
!wandb login

wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


In [2]:
subjects_test = all_subjects_intense[8:9]
subjects_train = [sub for sub in all_subjects_intense if sub not in subjects_test]

In [3]:
sweep_config = {
    'method': 'random',
    "early_terminate": {
          "type": "hyperband",
          "max_iter": 10,
          "s": 2
    },
    "metric": {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "last_layer": {"values": [64, 32, 16]},
        "batch_size": {"values": [64, 128, 256]},
        "dropout": {"values": [0.35, 0.5]}
    }
}

In [4]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        trainer_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\INTENSE\\normalized_subjects.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test,
            "wandb": True,
            "log": False,
            "learning_rate": 0.001,
            "batch_size": config.batch_size,
            "batch_size_test": 256,
            "freeze_embed": False,
            "dataset_ignore_subject_train": True,
            "dataset_ignore_subject_test": False,
            "lr_steps": 5,
            "decision_function": 0,
            "filter": [0, 1, 2, 3, 4, 5, 6, -1, -2, -3, -4, -5, -6],
            "weight_decay": 0,
            "adam": True,
            "number_steps": 200,
            "number_steps_testing": None,
            "number_steps_histogramm": None
        }

        model_embedding_hyperparameter = {
            "sigmoid": False,
            "dropout": config.dropout,
            "layers": [207, 512, 64]
        }

        model_classifier_hyperparameter = {
            "dropout": config.dropout,
            "layers": [64, config.last_layer],
            "head_type": 0
        }

        classifier_model = ClassificationModel(model_classifier_hyperparameter)
        embedding_model = EmbeddingsModel(model_embedding_hyperparameter)
        #_ = embedding_model.load_state_dict(torch.load("D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Results\models\embedding_models\model_abs_acc_89.40.pth"))
        siamese_model = SiameseModel(embedding_model, classifier_model, decision_function=trainer_hyperparameter["decision_function"])

        #filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])
        trainer = SiameseTrainerCombinationDataset(trainer_hyperparameter, siamese_model, device="cuda:0", filter=None)

        trainer.trainloop(10)

In [5]:
sweep_id = wandb.sweep(sweep_config, project="PainLevelShiftDetection_Combination_Dataset_SiameseNetwork")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: e70e57hf
Sweep URL: https://wandb.ai/svenk/PainLevelShiftDetection_Combination_Dataset_SiameseNetwork/sweeps/e70e57hf


In [6]:
wandb.agent(sweep_id, train, count=27)

wandb: Agent Starting Run: qtogwmm4 with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.5
wandb: 	last_layer: 16
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


accuracy,▁▁████▅▅██
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.48889
epoch,10


wandb: Agent Starting Run: gs23tsdc with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.35
wandb: 	last_layer: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,▁▄▅▇██▇▅▅▆
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.6
epoch,10


wandb: Agent Starting Run: 56xu7lcr with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.5
wandb: 	last_layer: 16
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: s36km6m7 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.35
wandb: 	last_layer: 32
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,▁▂▇█▅▆▄▆▆▆
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.64444
epoch,10


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: tt6l4vjh with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.35
wandb: 	last_layer: 64
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,▇▂▅▁▅█▅▇▆█
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.6
epoch,10


wandb: Agent Starting Run: tzoa9ria with config:
wandb: 	batch_size: 256
wandb: 	dropout: 0.5
wandb: 	last_layer: 64
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
